In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool")
import denoise
from model import *

In [3]:
data = pd.read_csv("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/rssi/rssi_for_4AP.csv")

In [4]:
data

,Label,AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi
0,11,-60.0,-68.0,-61.0,-59.0
1,11,-62.0,-70.0,-59.0,-59.0
2,11,-61.0,-69.0,-62.0,-55.0
3,11,-62.0,-69.0,-63.0,-55.0
4,11,-61.0,-70.0,-62.0,-55.0
...,...,...,...,...,...
24495,35,-66.0,-65.0,-61.0,-59.0
24496,35,-67.0,-64.0,-62.0,-59.0
24497,35,-65.0,-65.0,-61.0,-60.0
24498,35,-65.0,-65.0,-61.0,-58.0


In [5]:
class RSSIDataset(Dataset):
    def __init__(self, rssi_data, labels, augment=False, rssi_mask_prob=0.05):
        self.rssi_data = torch.tensor(rssi_data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)  # One-hot labels
        self.augment = augment
        self.rssi_mask_prob = rssi_mask_prob

    def __len__(self):
        return len(self.rssi_data)

    def __getitem__(self, idx):
        rssi = self.rssi_data[idx].clone()
        label = self.labels[idx]

        if self.augment:
            # --- RSSI 擴增：隨機遮蔽部分值 ---
            mask = torch.rand(rssi.shape) < self.rssi_mask_prob
            rssi[mask] = 0.0

        return rssi, label

In [28]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# 假設你已經有 df_merged_all
rssi_columns = ['AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi']

# 1. 填補 RSSI 缺值
data[rssi_columns] = data[rssi_columns].fillna(0)

# 2. Label 編碼（如果是整數，可以直接用）
labels = data['Label'].values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# 3. One-hot 編碼
num_classes = len(label_encoder.classes_)
labels_onehot = torch.eye(num_classes)[labels_encoded]

# 4. RSSI 資料轉為 float tensor
rssi_data = torch.tensor(data[rssi_columns].values, dtype=torch.float32)

# # 5. 分割資料
# X_train, X_val, y_train, y_val = train_test_split(
#     rssi_data, labels_onehot, test_size=0.2, random_state=42, stratify=labels_encoded
# )

# 6. 包裝成 Dataset
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [6]:
from sklearn.model_selection import train_test_split

# 取得原始的 index
indices = data.index

# 先切分 train 與 temp (train: 70%, temp: 30%)
train_idx, temp_idx = train_test_split(indices, test_size=0.3, random_state=42)

# 接著切分 temp 為 validation 與 test (各 10% 與 20%，因為 1/3 的 30% 為 10%)
val_idx, test_idx = train_test_split(temp_idx, test_size=1/3, random_state=42)

# 使用 .loc 根據切分的 index 取得各部分資料
rssi_train = data.loc[train_idx]
rssi_val   = data.loc[val_idx]
rssi_test  = data.loc[test_idx]

# 假設 one_hot_labels 也是與 amp 同 index 的 Series 或 DataFrame
#y_train = rp_labels.loc[train_idx]
#y_val   = rp_labels.loc[val_idx]

In [7]:
data.Label


0        11
1        11
2        11
3        11
4        11
         ..
24495    35
24496    35
24497    35
24498    35
24499    35
Name: Label, Length: 24500, dtype: int64

In [ ]:
int_labels = np.argmax(one_hot_labels, axis=1)

In [8]:
encoder = OneHotEncoder(sparse_output=False)

y_train_oh = encoder.fit_transform(np.array(rssi_train[['Label']]).reshape(-1, 1))
y_val_oh = encoder.transform(np.array(rssi_val[['Label']]).reshape(-1, 1))
y_test_oh = encoder.transform(np.array(rssi_test[['Label']]).reshape(-1, 1))

In [22]:
y_train_oh.shape

(17150, 49)

In [10]:
rssi_train.drop(columns=['Label'], inplace=True)
rssi_val.drop(columns=['Label'], inplace=True)
rssi_test.drop(columns=['Label'], inplace=True)

In [11]:
class RSSIDataset(Dataset):
    def __init__(self, rssi_data, labels, augment=False, rssi_mask_prob = 0.05):
        self.rssi_data = torch.tensor(rssi_data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)  # One-hot labels
        self.augment = augment
        self.rssi_mask_prob = rssi_mask_prob

    def __len__(self):
        return len(self.rssi_data)

    def __getitem__(self, idx):
        rssi = self.rssi_data[idx]
        label = self.labels[idx]

        if self.augment:
            # --- RSSI 擴增：隨機遮蔽部分值 ---
            mask = torch.rand(rssi.shape) < self.rssi_mask_prob
            rssi[mask] = 0.0

        return rssi, label

In [12]:
train_dataset = RSSIDataset(rssi_train.values, y_train_oh, augment=True)
val_dataset = RSSIDataset(rssi_val.values, y_val_oh, augment=False)
test_dataset = RSSIDataset(rssi_test.values, y_test_oh, augment=False)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
for train_input, train_label in train_loader:
    print(train_input.shape, train_label.shape)

torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 4]) torch.Size([32, 49])
torch.Size([32, 

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class RSSIClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RSSIClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 32)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [15]:
model = RSSIClassifier(4, num_classes=49).to(device)

In [16]:

dummy_rssi = torch.randn(1, 4).to(device)       # RSSI 輸入: (batch=1, 4-d)
output = model(dummy_rssi)
print("Output shape:", output.shape)  # 預期: (1, 49)

# 安裝 torchinfo (如果還沒安裝)
# pip install torchinfo

from torchinfo import summary
dummy_rssi = torch.randn(1, 4).to(device)
summary(model, input_data=dummy_rssi)


Output shape: torch.Size([1, 49])


Layer (type:depth-idx)                   Output Shape              Param #
RSSIClassifier                           [1, 49]                   --
├─Linear: 1-1                            [1, 32]                   160
├─Dropout: 1-2                           [1, 32]                   --
├─Linear: 1-3                            [1, 32]                   1,056
├─Dropout: 1-4                           [1, 32]                   --
├─Linear: 1-5                            [1, 49]                   1,617
Total params: 2,833
Trainable params: 2,833
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01

In [26]:
import torch.nn as nn
import torch.optim as optim

# 損失函數
criterion = nn.CrossEntropyLoss()

# 優化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 學習率調整器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

In [21]:
print(train_loader.dataset[0][0], train_loader.dataset[0][1])

tensor([-61., -66., -56., -52.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [27]:
import torch
import matplotlib.pyplot as plt

# 儲存最佳模型相關設定
best_val_loss = float('inf')
best_model_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth"

# 訓練參數
epochs = 200

# Early Stopping 參數
patience = 20
counter = 0  

# 紀錄訓練過程中的 loss 和 accuracy
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    # ---- 訓練階段 ----
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0

    for rssi_inputs, labels in train_loader:
        rssi_inputs = rssi_inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(rssi_inputs)

        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * rssi_inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        train_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_train_loss = train_loss / len(train_loader.dataset)
    train_acc = 100 * train_correct / total_train

    # ---- 驗證階段 ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for rssi_inputs, labels in val_loader:
            rssi_inputs = rssi_inputs.to(device)
            labels = labels.to(device)

            outputs = model(rssi_inputs)
            loss = criterion(outputs, torch.argmax(labels,dim=1))

            val_loss += loss.item() * rssi_inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            val_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_acc = 100 * val_correct / total_val

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # 儲存最佳模型
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ 儲存最佳模型 (Val Loss: {best_val_loss:.4f}) 至 {best_model_path}")
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"⏹️ Early stop at epoch {epoch+1}")
        break

print("✅ 訓練完成！")


Epoch [1/200] | Train Loss: 3.8918 | Train Acc: 2.26% | Val Loss: 3.8952 | Val Acc: 1.43%
✅ 儲存最佳模型 (Val Loss: 3.8952) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [2/200] | Train Loss: 3.8922 | Train Acc: 2.29% | Val Loss: 3.8952 | Val Acc: 1.43%
✅ 儲存最佳模型 (Val Loss: 3.8952) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [3/200] | Train Loss: 3.8917 | Train Acc: 2.27% | Val Loss: 3.8952 | Val Acc: 1.43%
✅ 儲存最佳模型 (Val Loss: 3.8952) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [4/200] | Train Loss: 3.8919 | Train Acc: 2.27% | Val Loss: 3.8951 | Val Acc: 1.43%
✅ 儲存最佳模型 (Val Loss: 3.8951) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [5/200] | Train Loss: 3.8917 | Train Acc: 2.31% | Val Loss: 3.8951 | Val Acc: 1.43%
Epoch [6/200] | Train Loss: 3.8920 | Train Acc: 2.

KeyboardInterrupt: 